In [ ]:
## Produce the smearing matrices & efficiencies using the intrinsic nue samples
# able to handle both FHC & RHC separate or combined

In [1]:
import sys

from scipy.stats import norm
from matplotlib.colors import LogNorm
import xsec_functions

import importlib
import uproot
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

importlib.reload(xsec_functions)
from xsec_functions import *

import selection_functions



In [2]:
importlib.reload(selection_functions)
from selection_functions import *

In [3]:
import awkward

In [4]:

fhc_intrinsic_pot = 2.37838E22
rhc_intrinsic_pot = 2.5345E22


In [5]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

path = '/uboone/data/users/kmiller/ntuples/run1/' #'/uboone/data/users/kmistry/work/MCC9/searchingfornues/'

In [6]:
# REPLACE WITH NUE INTRINSIC

# Run 1 FHC
FHC_NUE = 'neutrinoselection_filt_run1_overlay_intrinsic_v7'
#'ntuple_files_v7/neutrinoselection_filt_run1_overlay_intrinsic'


In [ ]:
# Run 3b RHC
RHC_NUE = 'ntuple_files_v7/neutrinoselection_filt_run3b_overlay_intrinsic'

In [7]:
fhc_nue = uproot.open(path+FHC_NUE+".root")[fold][tree]

In [ ]:
rhc_nue = uproot.open(path+RHC_NUE+".root")[fold][tree]

In [8]:
uproot_v = [fhc_nue]#,rhc_nue]

In [9]:

variables = [
    "selected", "nu_pdg", "shr_theta",
    "trk_score_v", 
    "category", 
    "shr_tkfit_dedx_Y",
    "ccnc", 
    "hits_ratio", "n_tracks_contained", 
    "NeutrinoEnergy2",
    "run","sub","evt",
    "CosmicIP",
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nproton", "nu_e", "n_showers_contained", "nu_purity_from_pfp", 
    "shr_phi", "trk_phi", "trk_theta", "trk_pid_chipr_v",
    "shr_score", 
    "trk_energy", "trk_energy_tot", "tksh_distance", "tksh_angle",
    "npi0", "topological_score",
    "shr_energy_tot_cali", 
    "nslice", "interaction",
    "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z", "contained_fraction",
    "true_nu_vtx_x", "true_nu_vtx_y" , "true_nu_vtx_z", 
    "npion", "shr_energy_cali", 
    "all_trk_energies", "all_trk_hits", "all_shr_energies", "all_shr_hits", ## for secondary shower / track 
    "shrmoliereavg", "shr_px", "shr_py", "shr_pz", 
    "shrsubclusters0","shrsubclusters1","shrsubclusters2", 
    "shr_tkfit_npointsvalid","shr_tkfit_npoints", "true_e_visible", "trkshrhitdist2", 
    "weightSplineTimesTune", "swtrig", "true_nu_px", "true_nu_py", "true_nu_pz", 
            "elec_e", "proton_e", "mc_px", "mc_py", "mc_pz", "elec_px", "elec_py", "elec_pz", 
            "swtrig_pre", "ppfx_cv", "mc_pdg", "weightsGenie", "weightsReint", 'weightsPPFX', 'opening_angle' ]



In [10]:
fhc_nue = fhc_nue.pandas.df(variables, flatten=False)

In [ ]:
rhc_nue = rhc_nue.pandas.df(variables, flatten=False)

In [12]:
df_v = [fhc_nue]#, rhc_nue]
#df_v = addAngles(df_v)

for i,df in enumerate(df_v):
    up = uproot_v[i]
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    df['trkpid'] = trk_llr_pid_v_sel
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    df['trkfit'] = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
    
    # check? 
    df['hadron_tot_e'] = df['true_e_visible'] - df['elec_e']# - 0.000511
    
    df['weightsPPFX'] = df['weightsPPFX']/1000
    


In [13]:
fhc_mc = fhc_nue.query('swtrig==1')

In [ ]:
rhc_mc = rhc_nue.query('swtrig==1')

In [14]:
in_fv_query = "10<=true_nu_vtx_x<=246 and -106<=true_nu_vtx_y<=106 and 10<=true_nu_vtx_z<=1026"
reco_in_fv_query = "10<=reco_nu_vtx_sce_x<=246 and -106<=reco_nu_vtx_sce_y<=106 and 10<=reco_nu_vtx_sce_z<=1026"

signal = '(nu_pdg==12 and ccnc==0 and nproton>0 and npion==0 and npi0==0)'
nue_CCother = '( ((nu_pdg==12 and ccnc==0) and (nproton==0 or npi0>0 or npion>0)) or (nu_pdg==-12 and ccnc==0 and (nproton==0 or npion>0 or npi0>0)) )'


In [15]:
fhc_infv = fhc_mc.query(in_fv_query+' and nu_purity_from_pfp>0.5')

fhc_infv['data_pot_scale'] = 2.0E20/fhc_intrinsic_pot #9.23E20/fhc_overlay_pot
fhc_infv['proj_pot_scale'] = 9.23E20/fhc_intrinsic_pot


/uboone/app/users/kmiller/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/uboone/app/users/kmiller/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
rhc_infv = rhc_mc.query(in_fv_query+' and nu_purity_from_pfp>0.5')

rhc_infv['data_pot_scale'] = 5.0E20/rhc_intrinsic_pot #11.95E20/rhc_overlay_pot
rhc_infv['proj_pot_scale'] = 11.95E20/rhc_intrinsic_pot

## Weights

In [16]:
# flux & GENIE weighting 

for i,df in enumerate([fhc_infv]):#, rhc_infv]):
    df.loc[ df['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
    df.loc[ np.isnan(df['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.

# total weight 
fhc_infv['totweight_data'] = fhc_infv['data_pot_scale']*fhc_infv['weightSplineTimesTune']*fhc_infv['ppfx_cv']
fhc_infv['totweight_proj'] = fhc_infv['proj_pot_scale']*fhc_infv['weightSplineTimesTune']*fhc_infv['ppfx_cv']
fhc_infv['totweight_intrinsic'] = fhc_infv['weightSplineTimesTune']*fhc_infv['ppfx_cv']



/uboone/app/users/kmiller/miniconda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/uboone/app/users/kmiller/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/uboone/app/users/kmiller/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [ ]:
rhc_infv['totweight_data'] = rhc_infv['data_pot_scale']*rhc_infv['weightSplineTimesTune']*rhc_infv['ppfx_cv']
rhc_infv['totweight_proj'] = rhc_infv['proj_pot_scale']*rhc_infv['weightSplineTimesTune']*rhc_infv['ppfx_cv']
rhc_infv['totweight_intrinsic'] = rhc_infv['weightSplineTimesTune']*rhc_infv['ppfx_cv']


In [17]:
fhc_infv['horn'] = 'fhc'
rhc_infv['horn'] = 'rhc'

/uboone/app/users/kmiller/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# nue & nuebar events within the true FV 
comb_infv = pd.concat([fhc_infv, rhc_infv], ignore_index=True)

## Linear Selection 

In [ ]:
# PRESELECTION 

reco_in_fv_query = "10<=reco_nu_vtx_sce_x<=246 and -106<=reco_nu_vtx_sce_y<=106 and 10<=reco_nu_vtx_sce_z<=1026"

PRE_QUERY = 'nslice==1'
PRE_QUERY += ' and ' + reco_in_fv_query
PRE_QUERY +=' and contained_fraction>0.9'
PRE_QUERY += ' and n_showers_contained>0'
PRE_QUERY += ' and n_tracks_contained>0'

In [ ]:
# FHC selection

FHC_SEL_QUERY = PRE_QUERY
FHC_SEL_QUERY += ' and shr_energy_tot_cali>0.07'

FHC_SEL_QUERY += ' and shr_score<0.125'
FHC_SEL_QUERY += ' and shrmoliereavg < 8'
FHC_SEL_QUERY += ' and trkpid<0'

FHC_SEL_QUERY += ' and n_showers_contained == 1'
FHC_SEL_QUERY += ' and shr_tkfit_dedx_Y<4'
FHC_SEL_QUERY += ' and tksh_distance<5'
#FHC_SEL_QUERY += ' and -0.8<tksh_angle<0.8'

fhc_sel = fhc_infv.query(FHC_SEL_QUERY)



In [ ]:
# RHC Selection 

RHC_SEL_QUERY = PRE_QUERY
RHC_SEL_QUERY += ' and shr_energy_tot_cali>0.07'

RHC_SEL_QUERY += ' and shr_score<0.125'
RHC_SEL_QUERY += ' and shrmoliereavg < 8'
RHC_SEL_QUERY += ' and trkpid<0'

RHC_SEL_QUERY += ' and n_showers_contained == 1'
RHC_SEL_QUERY += ' and shr_tkfit_dedx_Y<4'
RHC_SEL_QUERY += ' and tksh_distance<4'
RHC_SEL_QUERY += ' and -0.8<tksh_angle<0.8'

rhc_sel = rhc_infv.query(RHC_SEL_QUERY)

In [ ]:
comb_sel = pd.concat([fhc_sel, rhc_sel], ignore_index=True)

### Opening Angle

In [ ]:
signal = '(nu_pdg==12 and ccnc==0 and nproton>0 and npion==0 and npi0==0 and -0.8<opening_angle<0.8)'

In [ ]:
# distribution of generated signal events across opening angle 

fig = plt.figure(figsize=(8, 5))

gen = plt.hist(fhc_infv.query(signal)['opening_angle'], 20, color='orange', range=[-1, 1])

plt.xlabel("cos $\\theta$", fontsize=15)
plt.ylabel("# of generated signal events", fontsize=15)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.title("Signal: PDG==12, CC, N>0 protons, 0 pions, Op.Angle < +/-0.8 ", fontsize=14)
#plt.savefig("/uboone/data/users/kmiller/searchingfornues_v33/v08_00_00_33/plots/fhc/.pdf", transparent=True)
plt.show()

In [ ]:
# distribution of selected signal events across opening angle 

fig = plt.figure(figsize=(8, 5))

sel = plt.hist(fhc_sel.query(signal)['opening_angle'], 20, color='orange', range=[-1, 1])

plt.xlabel("cos $\\theta$", fontsize=15)
plt.ylabel("# of selected signal events", fontsize=15)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.title("Signal: PDG==12, CC, N>0 protons, 0 pions, Op.Angle < +/-0.8\nLinear Selection w/o tksh_angle Cut ", fontsize=14)

plt.show()

In [ ]:
# efficiency across opening angle (current linear selection)

eff = [x/y for x, y in zip(sel[0], gen[0])]
eff_err = [ np.sqrt( (eff[i]*(1-eff[i]))/gen[0][i] ) for i in range(len(eff))]

bincenters = 0.5*(gen[1][1:]+gen[1][:-1])
binwidth = [abs(gen[1][1]-gen[1][0])/2 for x in range(len(bincenters))]

fig = plt.figure(figsize=(8, 5))

plt.errorbar(bincenters, eff, xerr=binwidth, yerr=eff_err, fmt='o', color='seagreen', ecolor='seagreen', markersize=3) 

plt.xlabel("cos $\\theta$", fontsize=15)
plt.ylabel("Efficiency", fontsize=15)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.ylim(0, 0.25)

plt.title("Signal: PDG==12, CC, N>0 protons, 0 pions\nLinear Selection with tksh_angle Cut ", fontsize=14)
plt.grid(linestyle=':')
plt.show()



In [ ]:
# efficiency across opening angle (lin sel without tksh_angle cut)


eff = [x/y for x, y in zip(sel[0], gen[0])]
eff_err = [ np.sqrt( (eff[i]*(1-eff[i]))/gen[0][i] ) for i in range(len(eff))]

bincenters = 0.5*(gen[1][1:]+gen[1][:-1])
binwidth = [abs(gen[1][1]-gen[1][0])/2 for x in range(len(bincenters))]

fig = plt.figure(figsize=(8, 5))

plt.errorbar(bincenters, eff, xerr=binwidth, yerr=eff_err, fmt='o', color='seagreen', ecolor='seagreen', markersize=3) 

plt.xlabel("cos $\\theta$", fontsize=15)
plt.ylabel("Efficiency", fontsize=15)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.ylim(0, 0.25)

plt.title("Signal: PDG==12, CC, N>0 protons, 0 pions, \nLinear Selection w/o tksh_angle Cut ", fontsize=14)
plt.grid(linestyle=':')
plt.show()

In [ ]:
# efficiency across opening angle (lin sel without tksh_angle cut & modified signal def )


# efficiency across opening angle (lin sel without tksh_angle cut)


eff = [x/y for x, y in zip(sel[0], gen[0])]
eff_err = [ np.sqrt( (eff[i]*(1-eff[i]))/gen[0][i] ) for i in range(len(eff))]

bincenters = 0.5*(gen[1][1:]+gen[1][:-1])
binwidth = [abs(gen[1][1]-gen[1][0])/2 for x in range(len(bincenters))]

fig = plt.figure(figsize=(8, 5))

plt.errorbar(bincenters, eff, xerr=binwidth, yerr=eff_err, fmt='o', color='seagreen', ecolor='seagreen', markersize=3) 

plt.xlabel("cos $\\theta$", fontsize=15)
plt.ylabel("Efficiency", fontsize=15)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.xlim(-1, 1)
plt.ylim(0, 0.25)

plt.title("Signal: PDG==12, CC, N>0 protons, 0 pions, Op.Angle < +/-0.8\nLinear Selection w/o tksh_angle Cut ", fontsize=14)
plt.grid(linestyle=':')
plt.show()

## BDT selection

In [18]:
# rather than re-train the BDT, we need to import the same model used in selection.ipynb

import xgboost as xgb

In [19]:
bdt_model = xgb.Booster({'nthread': 4})  # init model

In [20]:
bdt_model.load_model('bdt_model_feb2021.model')

In [21]:
BDT_PRE_QUERY = 'nslice==1'
BDT_PRE_QUERY += ' and ' + reco_in_fv_query
BDT_PRE_QUERY +=' and contained_fraction>0.9'
BDT_PRE_QUERY += ' and n_tracks_contained>0'
BDT_PRE_QUERY += ' and n_showers_contained==1'
BDT_PRE_QUERY += ' and shr_energy_tot_cali>0.07'

In [22]:
BDT_LOOSE_CUTS = BDT_PRE_QUERY
BDT_LOOSE_CUTS += ' and shr_score<0.3'
BDT_LOOSE_CUTS += ' and trkpid<0.35'
BDT_LOOSE_CUTS += ' and shrmoliereavg<15'
BDT_LOOSE_CUTS += ' and shr_tkfit_dedx_Y<7'
BDT_LOOSE_CUTS += ' and tksh_distance<12'

In [24]:
# apply the saved model to our df 

varlist = [
    "shr_score", "shrmoliereavg", "trkpid",
    "n_showers_contained", "shr_tkfit_dedx_Y", "tksh_distance",
    "tksh_angle", "subcluster", "trkshrhitdist2"]

# apply cuts
bdt_results = fhc_infv.copy()
bdt_results = bdt_results.query(BDT_LOOSE_CUTS)

# clean datasets 
for column in varlist:
        bdt_results.loc[(bdt_results[column] < -1.0e37) | (bdt_results[column] > 1.0e37), column] = np.nan
    
# create testing dmatrix 
bdt_results_test = xgb.DMatrix(data=bdt_results[varlist])
    
# apply the bdt selection
preds = bdt_model.predict(bdt_results_test)

# add columns for plotting 
bdt_results['BDT_score'] = preds

In [26]:
selected = bdt_results.query('BDT_score>0.575')

### Opening Angle

In [ ]:
# compute the magnitude of all MC Particles
def mc_p(df):
    
    p_all = []
    for index, row in df.iterrows():
        p_row = []
        for i in range(len(row['mc_pdg'])): 
            p = math.sqrt( row['mc_px'][i]*row['mc_px'][i] + row['mc_py'][i]*row['mc_py'][i] + row['mc_pz'][i]*row['mc_pz'][i] )
            p_row.append(p)
        
        p_all.append( p_row )
    
    df['mc_p'] = p_all 
    
    return df 


In [ ]:
# now get the proton with the highest momentum 
# use this to construct the truth opening angle 

def tksh_angle_truth(df): 
    
    tksh_angle_truth = []
    for index, row in df.iterrows(): 

        proton_max_p = 0
        proton_max_p_idx = 0
    
        # for each index in the PDG list 
        for i in range(len(row['mc_pdg'])): 
        
            # check if it is a proton 
            if row['mc_pdg'][i]==2212: 
                # check if it is max
                if row['mc_p'][i]>proton_max_p: 
                    # if so, replace
                    proton_max_p = row['mc_p'][i] 
                    proton_max_p_idx = i 
                
        # now use the leading proton index to compute the opening angle 
        proton_p = [ row['mc_px'][i], row['mc_py'][i], row['mc_pz'][i] ]
    
        # and construct electron vector (should be norm'd already)
        elec_p = [ row['elec_px'], row['elec_py'], row['elec_pz'] ]
        elec_p_mag = np.sqrt((elec_p[0]*elec_p[0])+(elec_p[1]*elec_p[1])+(elec_p[2]*elec_p[2]))
    
        # opening angle 
        cos = np.dot(proton_p, elec_p) / (proton_max_p*elec_p_mag)
        if proton_max_p==0 or elec_p_mag==0: 
            tksh_angle_truth.append(np.nan)
        
        else: 
            tksh_angle_truth.append(cos)
            
    df['opening_angle'] = tksh_angle_truth 
    
    return df
 

In [ ]:
fhc_sel = mc_p(fhc_sel)
rhc_sel = mc_p(rhc_sel)

In [ ]:
fhc_sel = tksh_angle_truth(fhc_sel)
rhc_sel = tksh_angle_truth(rhc_sel)

In [ ]:
fhc_sel.query('opening_angle>1')['opening_angle']

In [ ]:
fhc_sel.query('opening_angle<-1')['opening_angle']

In [ ]:
plt.hist([fhc_sel.query(signal)['opening_angle'], fhc_sel.query(signal)['tksh_angle']], 5,
        histtype='step', label=['true', 'reco'], weights=[fhc_sel.query(signal)['totweight_proj'], 
                                                          fhc_sel.query(signal)['totweight_proj']])
plt.title('Opening Angle')
plt.xlabel('cos $\\theta$')
plt.ylabel('$\\nu$ \ 9.23E20 POT')
plt.legend()

#plt.savefig('/uboone/data/users/kmiller/searchingfornues_v33/v08_00_00_33/plots/fhc/opening_angle_wide.pdf', 
#         transparent=True, bbox_inches='tight')
plt.show()



### scattering angle 

In [ ]:
# construct the true scattering angle 

def true_scatangle(df): 
    # length of the neutrino momentum vector
    nu_p_length = np.sqrt((df['true_nu_px']*df['true_nu_px'] + df['true_nu_py']*df['true_nu_py'] + df['true_nu_pz']*df['true_nu_pz']))
    df['true_nu_pmag'] = nu_p_length

    # use the normalized neutrino & electron momentum vectors (both in det coordinates) to construct the angle
    df['true_scatangle'] = ((df['true_nu_px']/df['true_nu_pmag'])*df['elec_px'])+((df['true_nu_py']/df['true_nu_pmag'])*df['elec_py'])+((df['true_nu_pz']/df['true_nu_pmag'])*df['elec_pz'])
   
    return df

In [ ]:
# construct the reco scattering angle 

def reco_scatangle(df): 


    # step 1: construct the neutrino direction vector (interaction vertex - beam target) in detector coordinates
    NuMITargetPos = fromBeamtoDet(0, 0, 0, False)

    # neutrino direction vector - detector coordinates, reconstructed, normalized
    
    nuX = df['reco_nu_vtx_sce_x'] - NuMITargetPos[0]
    nuY = df['reco_nu_vtx_sce_y'] - NuMITargetPos[1]
    nuZ = df['reco_nu_vtx_sce_z'] - NuMITargetPos[2]

    nu_dir_mag = np.sqrt((nuX*nuX)+(nuY*nuY)+(nuZ*nuZ))


    # step 2: normalize the neutrino direction vector and the reco shower momentum vector 
    nuX = nuX/nu_dir_mag
    nuY = nuY/nu_dir_mag
    nuZ = nuZ/nu_dir_mag

    shr_px = df['shr_px']
    shr_py = df['shr_py']
    shr_pz = df['shr_pz']

    shr_p_mag = np.sqrt((shr_px*shr_px)+(shr_py*shr_py)+(shr_pz*shr_pz))

    shr_px = shr_px/shr_p_mag
    shr_py = shr_py/shr_p_mag
    shr_pz = shr_pz/shr_p_mag

    # step 3: take the dot product between them 

    df['reco_scatangle'] = ( (nuX*shr_px)+(nuY*shr_py)+(nuZ*shr_pz) )
    
    return df



In [ ]:
fhc_sel = true_scatangle(fhc_sel)
rhc_sel = true_scatangle(rhc_sel)

In [ ]:
fhc_sel = reco_scatangle(fhc_sel)
rhc_sel = reco_scatangle(rhc_sel)

In [ ]:
from matplotlib.colors import LogNorm

In [ ]:
# angle off the beamline - numi coordinates (degrees)

plt.figure(figsize=(8, 5))

plt.hist([fhc_infv['thbeam'], fhc_infv['phibeam']], 
          40, histtype='step', weights=[fhc_infv['totweight_proj'], fhc_infv['totweight_proj']], 
          label=['$\\theta$ (off beamline)', '$\\phi$ (XY plane)'])
plt.xlabel('Angle [deg]', fontsize=14)
plt.ylabel('$\\nu$ / 9.23E20 POT', fontsize=14)
plt.yscale("log")
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.title('FHC events in FV (overlay only)', fontsize=14)
plt.legend(fontsize=13)
plt.savefig('/uboone/data/users/kmiller/searchingfornues_v33/v08_00_00_33/plots/fhc/thetavsphi.pdf', 
         transparent=True, bbox_inches='tight')
plt.show()

In [ ]:

plt.figure(figsize=(8, 6))

plt.hist2d(fhc_infv['nu_e'], fhc_infv['thbeam'], 
           40, cmin=0.0000001, range=[[0, 5], [0, 140]], weights=fhc_infv['totweight_proj'])#, norm=LogNorm())
plt.xlabel('true neutrino energy [GeV]', fontsize=14)
plt.ylabel('true angle off beamline [deg]', fontsize=14)
#plt.ylabel('$\\nu$ / 9.23E20 POT', fontsize=14)
#plt.yscale("log")
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
cbar = plt.colorbar()
#cbar.set_label('$\\nu$ / 11.95E20 POT (RHC projected)', fontsize=14)
cbar.ax.tick_params(labelsize=14)
plt.title("FHC events in FV (overlay only)", fontsize=14)
#plt.savefig('/uboone/data/users/kmiller/searchingfornues_v33/v08_00_00_33/plots/rhc/rhc_infv2d.pdf', 
#           transparent=True, bbox_inches='tight')
plt.show()



plt.figure(figsize=(8, 6))

plt.hist2d(fhc_sel['nu_e'], fhc_sel['thbeam'], 
           40, cmin=0.0000001, range=[[0, 5], [0, 140]], weights=fhc_sel['totweight_proj'])
          #, norm=LogNorm())
plt.xlabel('true neutrino energy [GeV]', fontsize=14)
plt.ylabel('true angle off beamline [deg]', fontsize=14)
#plt.ylabel('$\\nu$ / 9.23E20 POT', fontsize=14)
#plt.yscale("log")
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
plt.title("FHC selected events in FV (overlay only)", fontsize=14)
#cbar.set_label('$\\nu$ / 11.95E20 POT (RHC projected)', fontsize=14)
#plt.savefig('/uboone/data/users/kmiller/searchingfornues_v33/v08_00_00_33/plots/rhc/rhc_sel2d.pdf', 
#           transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
# distribution of selected signal 

plt.hist(fhc_sel.query(signal)['theta'], 10, range=[0, 3], histtype='step', color='orange')
plt.xlabel('cos($\\theta$)', fontsize=14)
plt.title("angle between incoming & outgoing leptons - what is this?")
plt.show()

In [ ]:
# smearing 1D

plt.hist([fhc_sel.query(signal)['true_scatangle'], fhc_sel.query(signal)['reco_scatangle']], 
         20, histtype='step', weights=[fhc_sel.query(signal)['totweight_proj'], 
                                      fhc_sel.query(signal)['totweight_proj']], 
        label=['true', 'reco'])
plt.xlabel('cos($\\theta$)', fontsize=14)
plt.legend(loc=2)
plt.title("angle between incoming & outgoing leptons")
#plt.savefig('/uboone/data/users/kmiller/searchingfornues_v33/v08_00_00_33/plots/fhc/fhc_scatangle.pdf', 
#           transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
# does this assume the neutrino is coming from BNB direction? 

plt.hist([fhc_sel.query(signal)['theta'], fhc_sel.query(signal)['shr_theta']],
         20, range=[0, 3], histtype='step', label=['true', 'reco'])
plt.xlabel('cos($\\theta$)', fontsize=14)
plt.title("angle between incoming & outgoing leptons")
plt.legend()
#plt.savefig('/uboone/data/users/kmiller/searchingfornues_v33/v08_00_00_33/plots/fhc/fhc_theta.pdf', 
#           transparent=True, bbox_inches='tight')
plt.show()

### smearing & binning

In [ ]:
#reco = 'n_tracks_contained'
#true = 'nproton'
#bins = [1, 2, 3, 7]

# total or leading? Uh oh  
#true = 'proton_e'
#reco = 'trk_energy' #trk_energy_tot ?? 
#bins = [ 0.04, 0.24, 0.44, 1.26 ]

#true = 'true_e_visible'
#reco = 'NeutrinoEnergy2'
#bins = [0.19, .4, .61, .82, 1.10, 1.45, 4]

reco = 'shr_energy_cali'
true = 'elec_e'
bins = [0.09, 0.35, 0.65, 1.05, 1.5, 3]



In [ ]:
save = True

In [ ]:
n, b, p = plt.hist([ selected.query(signal)[true], selected.query(nue_CCother)[true]], range=[0, 4],
                        stacked=True, color=['orange', 'limegreen'], label=['signal', 'nue CC other'])
plt.ylim(0, 6000)
plt.legend()
plt.show()

In [ ]:
# from here on out we only work with signal 
selected_signal = selected.query(signal)

In [ ]:
# detector resolution - FHC, RHC, BOTH 

#true_reco_res(true, reco, fhc_sel, 'fhc', 90, save)
#true_reco_res(true, reco, rhc_sel, 'rhc', 90, save)

true_reco_res(true, reco, selected_signal, 'both', 5000, save)

In [ ]:
# min reco selected
min(list(selected.query(signal)[reco]))

In [ ]:
# min true  selected
min(list(selected.query(signal)[true]))

In [ ]:
# smearing - FHC, RHC, BOTH 

#smear_matrix(true, reco, bins, fhc_sel, 'fhc', 55, 'electron energy [GeV]', save)
#smear_matrix(true, reco, bins, rhc_sel, 'rhc', 55, 'electron energy [GeV]', save)

smear_matrix(true, reco, bins, selected_signal, 'both', 2500, 'electron energy [GeV]', save)

In [ ]:
# efficiency - FHC & RHC 

#truth_and_eff(true, bins, fhc_infv, fhc_sel, 'fhc', 100, 'True Electron Energy [GeV]', save)
#truth_and_eff(true, bins, fhc_infv, fhc_sel, 'rhc', 100, 'True Electron Energy [GeV]', save)



In [ ]:
comb_truth_and_eff(true, bins, comb_infv, selected_signal, 'electron energy [GeV]', save)

In [ ]:
print('total events = '+str(sum(selected_signal['totweight_data'])))

### FHC or RHC

In [ ]:
true = 'elec_e'
reco = 'shr_energy_cali'

horn = 'fhc'

if horn=='fhc': 
    gen_df = fhc_infv.query(signal)
    sel_df = fhc_sel.query(signal)
    
elif horn=='rhc': 
    gen_df = rhc_infv.query(signal)
    sel_df = rhc_sel.query(signal)

# shr energy
bins = [0.09, .35, .65, 1.05, 1.5, 3]

# reco neutrino energy 
#bins = [ 0.19, .4, .61, .82, 1.03, 1.4, 2, 4 ]

# track mult
#bins = [1, 2, 3, 7]




In [ ]:
save = True

In [ ]:
true_reco_res(true, reco, sel_df, horn, 90, save)

In [ ]:
truth_and_eff(true, bins, gen_df, sel_df, horn, 100, 'Visible Energy [GeV]', save)

In [ ]:
smear_matrix(true, reco, bins, sel_df, horn, 35, 'neutrino energy [GeV]', save)


In [ ]:
# min reco  selected
min(list(sel_df[reco]))

In [ ]:
# min true  selected
min(list(sel_df[true]))

### FHC & RHC

In [ ]:
comb_sel = pd.concat([fhc_sel, rhc_sel], ignore_index=True)
comb_infv = pd.concat([fhc_infv, rhc_infv], ignore_index=True)

In [ ]:
df_sel = comb_sel.query(signal)
df_gen = comb_infv.query(signal)

In [ ]:
true = 'true_e_visible'
reco = 'NeutrinoEnergy2'


# shr energy
#bins = [0.09, .35, .65, 1.05, 1.5, 3]

# reco energy 
bins = [ 0.19, .4, .61, .82, 1.03, 1.4, 2, 4 ]

# track mult
#bins = [1, 2, 3, 7]

# hadron energy 
#bins = [.04, .23, .42, 1.3]




In [ ]:
save = True

In [ ]:
plt.figure(figsize=(8, 5))

plt.hist(df_sel['true_e_visible'], 16, range=[0, 4],histtype='step', label='true visible energy', 
         color='limegreen', weights=df_sel['totweight_proj'])
plt.hist(df_sel['nu_e'], 16, range=[0, 4], histtype='step', label='true $\\nu$ energy', 
         color='blue', weights=df_sel['totweight_proj'])

plt.legend(fontsize=13)
plt.xlabel("True Energy [GeV]", fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylim(0, 175)
plt.ylabel("$\\nu$ / 2.12E21 POT", fontsize=14)
#plt.savefig("/uboone/data/users/kmiller/searchingfornues_v33/v08_00_00_33/plots/both/neutrino_visible_energy.pdf", 
#            transparent=True, bbox_inches='tight') 
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))

plt.hist(df_sel['true_e_visible'], 16, range=[0, 4],histtype='step', label='true visible energy', 
         color='limegreen', weights=df_sel['totweight_proj'])
plt.hist(df_sel['NeutrinoEnergy2']/1000, 16, range=[0, 4], histtype='step', 
         label='reco energy', 
         color='red', weights=df_sel['totweight_proj'])

plt.legend(fontsize=13)
plt.xlabel("True Energy [GeV]", fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylim(0, 175)
plt.ylabel("$\\nu$ / 2.12E21 POT", fontsize=14)
#plt.savefig("/uboone/data/users/kmiller/searchingfornues_v33/v08_00_00_33/plots/both/visible_reco_energy.pdf", 
#            transparent=True, bbox_inches='tight') 
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))

plt.hist(df_sel['nu_e'], 16, range=[0, 4],histtype='step', label='true $\\nu$ energy', 
         color='blue', weights=df_sel['totweight_proj'])
plt.hist(df_sel['NeutrinoEnergy2']/1000, 16, range=[0, 4], histtype='step', 
         label='reco energy', 
         color='red', weights=df_sel['totweight_proj'])

plt.legend(fontsize=13)
plt.xlabel("True Energy [GeV]", fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylim(0, 175)
plt.ylabel("$\\nu$ / 2.12E21 POT", fontsize=14)
#plt.savefig("/uboone/data/users/kmiller/searchingfornues_v33/v08_00_00_33/plots/both/neutrino_reco_energy.pdf", 
#            transparent=True, bbox_inches='tight') 
plt.show()

In [ ]:
true_reco_res(true, reco, df_sel, 'both', 325, save)

In [ ]:
# normalized to data POT, but made from overlay POT

smear_matrix(true, reco, bins, df_sel, 'both', 350, 'hadronic energy [GeV]', save)